In [38]:
# Phcli Jupyter Python Template
# 
# 使用手册：
# 1. 请将全局变量定义在第一个输入区内
# 2. Phcli 会自动在第二个输入区初始化 Spark Session
# 3. 所有 print 会在 phcli maxauto dag 后自动转为 logger.debug() 方法
# 4. 请在第三个输入区开始编码，phcli maxauto dag 后会全部归类为一个方法


# Config defined in here

############## == config == ###################
job_name = "qi"
job_runtime = "python3"
job_command = "submit"
job_timeout = 10.0
############## == config == ###################


# Variables defined in here

############## == input args == ###################
a = 123
b = 456
############## == input args == ###################

############## == output args == ###################
c = 'abc'
d = 'def'
############## == output args == ###################

In [39]:
# Initialize the Spark Session
# YARN URL: http://161.189.223.227:8088/cluster
import os
from pyspark.sql import SparkSession, functions as F

# prepare
spark = SparkSession.builder \
    .master("yarn") \
    .appName("mzhang write test.qi in jupyter using python3") \
    .config("spark.driver.cores", "1") \
    .config("spark.driver.memory", "4g") \
    .config("spark.executor.cores", "1") \
    .config("spark.executor.memory", "4g") \
    .config("spark.executor.instances", "1") \
    .config('spark.sql.codegen.wholeStage', False) \
    .enableHiveSupport() \
    .getOrCreate()


access_key = os.getenv("AWS_ACCESS_KEY_ID")
secret_key = os.getenv("AWS_SECRET_ACCESS_KEY")
print(access_key)
print(secret_key)
if access_key:
    spark._jsc.hadoopConfiguration().set("fs.s3a.access.key", access_key)
    spark._jsc.hadoopConfiguration().set("fs.s3a.secret.key", secret_key)
    spark._jsc.hadoopConfiguration().set("com.amazonaws.services.s3.enableV4", "true")
    spark._jsc.hadoopConfiguration().set("fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    spark._jsc.hadoopConfiguration().set("fs.s3a.endpoint", "s3.cn-northwest-1.amazonaws.com.cn")

None
None


In [ ]:
spark.sql("SHOW tables").show()

In [7]:
# spark.read.parquet("s3a://ph-stream/common/public/universe/0.0.1").show()

+----------+--------------+------------------+------------+------------+------------+----------+-------+----------+------------+---------+--------+---------------------------+------------+------------+---------+--------------+-----------+-------------+--------+---------------+------------+------------+---------+----------------+------------------+-------------------+--------------+-------------------+--------------------+-----------------------+--------------------+----------+--------+------------+-------------+--------------+----------------+-----------+----------------+-----------------+--------------------+------------------------+----------+------------+------------+------------+--------+---------------+--------------+-------------------------------+------------+--------------+---------+--------------------+------------+----------+-------+
|ADMIS_TIME|ANNU_DIAG_TIME|           AREA_MU|   AREA_SQ_M| AREA_STRUCT|AUTH_BED_NUM|BED_INCOME|BED_NUM|BID_SAMPLE|        CITY|CITY_TIER| COMPA

In [8]:
df_chc = spark.read.parquet("s3a://ph-max-auto/2020-08-11/data_matching/refactor/data/CHC/*")

In [9]:
df_standard = spark.read.parquet("s3a://ph-stream/common/public/prod/0.0.21/")

In [10]:
df_standard.count()

65875

In [12]:
df_standard_cols = df_standard.columns

In [13]:
len(df_standard_cols)

45

In [14]:
df_standard_cols

['PACK_ID',
 'MOLE_NAME_EN',
 'MOLE_NAME_CH',
 'PROD_DESC',
 'PROD_NAME_CH',
 'CORP_NAME_EN',
 'CORP_NAME_CH',
 'MNF_NAME_EN',
 'MNF_NAME_CH',
 'PCK_DESC',
 'DOSAGE',
 'SPEC',
 'SPEC_valid_digit',
 'SPEC_valid_unit',
 'SPEC_gross_digit',
 'SPEC_gross_unit',
 'PACK',
 'NFC1',
 'NFC1_NAME',
 'NFC1_NAME_CH',
 'NFC12',
 'NFC12_NAME',
 'NFC12_NAME_CH',
 'NFC123',
 'NFC123_NAME',
 'CORP_ID',
 'MNF_TYPE',
 'MNF_TYPE_NAME',
 'MNF_TYPE_NAME_CH',
 'MNF_ID',
 'ATC1_CODE',
 'ATC1_DESC',
 'ATC2_CODE',
 'ATC2_DESC',
 'ATC3_CODE',
 'ATC3_DESC',
 'ATC4_CODE',
 'ATC4_DESC',
 'ATC1',
 'ATC2',
 'ATC3',
 'ATC4',
 'PckLaunchDate',
 'PrdLaunchDate',
 'REMARK']

In [16]:
df_standard.select(['DOSAGE','SPEC','SPEC_valid_digit','SPEC_valid_unit','SPEC_gross_digit','SPEC_gross_unit','NFC1','NFC1_NAME_CH']).show(500)

+------------+-----------+----------------+---------------+----------------+---------------+----+------------+
|      DOSAGE|       SPEC|SPEC_valid_digit|SPEC_valid_unit|SPEC_gross_digit|SPEC_gross_unit|NFC1|NFC1_NAME_CH|
+------------+-----------+----------------+---------------+----------------+---------------+----+------------+
|      注射液|  20% 250ML|         50000.0|             MG|           250.0|             ML| F--|    注射用药|
|      注射液|  20% 500ML|        100000.0|             MG|           500.0|             ML| F--|    注射用药|
|      注射液|  20% 100ML|         20000.0|             MG|           100.0|             ML| F--|    注射用药|
|      注射液|  20% 250ML|         50000.0|             MG|           250.0|             ML| F--|    注射用药|
|      注射液|  10% 100ML|         10000.0|             MG|           100.0|             ML| F--|    注射用药|
|      注射液|  10% 250ML|         25000.0|             MG|           250.0|             ML| F--|    注射用药|
|      注射液|  20% 250ML|         50000.0|   

In [17]:
from pyspark.sql.functions import count

In [23]:
df_standard.groupBy("DOSAGE").agg(count("DOSAGE")).show(100)

+--------------------------+-------------+
|                    DOSAGE|count(DOSAGE)|
+--------------------------+-------------+
|                阴道泡腾片|          205|
|                  阴道乳液|            1|
|                    滴耳剂|           69|
|                      散剂|          781|
|                      片剂|        12486|
|                    凝胶剂|          293|
|                  阴道乳膏|           26|
|                    眼膏剂|          116|
|                    含漱液|           51|
|                      糊剂|            6|
|卡式瓶(笔芯)(慈善援助项目)|            1|
|                    血糖仪|          217|
|              泡腾片(橙味)|            4|
|                    口崩片|           97|
|                    灌肠剂|          156|
|                      锭剂|           15|
|                  复合包装|            9|
|                    喷雾剂|          105|
|                      胶囊|         7476|
|                  缓释胶囊|          383|
|                  干混悬剂|           46|
|              粉针剂(冻干)|            8|
|    

In [26]:
# df_standard_pen_dosage = df_standard.where(df_standard.SPEC_gross_unit == '喷').select('DOSAGE')

In [27]:
# df_standard_pen_dosage.groupBy("DOSAGE").agg(count('DOSAGE')).show()

+----------+-------------+
|    DOSAGE|count(DOSAGE)|
+----------+-------------+
|    喷雾剂|            5|
|干粉吸入剂|            6|
|    粉雾剂|            6|
|    吸入剂|            4|
|吸入粉雾剂|           10|
|    鼻喷剂|           48|
|    气雾剂|           87|
+----------+-------------+



In [28]:
# df_standard_MG = df_standard.where(df_standard.SPEC_gross_unit == 'MG').select('DOSAGE')

In [31]:
# df_standard_MG.groupBy("DOSAGE").agg(count('DOSAGE')).show(500)

+--------------+-------------+
|        DOSAGE|count(DOSAGE)|
+--------------+-------------+
|          散剂|          143|
|        凝胶剂|          258|
|          片剂|           47|
|      阴道乳膏|           24|
|        眼膏剂|           94|
|          糊剂|            5|
|        灌肠剂|            4|
|      复合包装|            2|
|        喷雾剂|           14|
|          胶囊|           27|
|      干混悬剂|            1|
|          糖浆|            4|
|        乳膏剂|          964|
|        乳胶剂|            5|
|          搽剂|            1|
|          洗剂|           13|
|      阴道软膏|            2|
|          贴剂|            9|
|        溶液剂|            2|
|          膜剂|            2|
|      阴道凝胶|           23|
|        糖衣片|            5|
|        口服液|            8|
|      泡腾颗粒|           13|
|        泡腾片|            1|
|          颗粒|            1|
|        舌下片|            2|
|        注射液|            8|
|      巴布膏剂|            7|
|      薄膜衣片|           10|
|          蜡棒|            2|
|        粉针剂|          285|
|       

In [33]:
# df_standard_new = spark.read.csv("s3a://ph-stream/common/public/prod/0.0.22/",header=True)

In [34]:
# df_standard_new.count()

66492

In [40]:
df_standard_new = spark.read.parquet("s3a://ph-stream/common/public/prod/0.0.22")

AnalysisException: Unable to infer schema for Parquet. It must be specified manually.;